DistilBert for Emotion Classification

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from datasets import load_dataset
import numpy as np
emotion_dataset = load_dataset("emotion")
print(emotion_dataset)
import pandas as pd
emotion_dataset.set_format(type="pandas")
train = emotion_dataset["train"][:]
train.info()
df = train
df = df.reset_index(drop=True)
df.info()

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16000 non-null  object
 1   label   16000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 250.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16000 non-null  object
 1   label   16000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 250.1+ KB


In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import word_tokenize


In [9]:
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text

    return text

In [10]:
df['text'] = df['text'].apply(clean_text)
df.head(15)

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
5,ive been feeling a little burdened lately wasn...,0
6,ive been taking or milligrams or times recomme...,5
7,i feel as confused about life as a teenager or...,4
8,i have been with petronas for years i feel tha...,1
9,i feel romantic too,2


a classification label, with possible values including sadness (0), joy (1), love (2), anger (3), fear (4), surprised (5)

In [11]:
#check distribution of data based on labels
print("Distribution of data based on labels: ",df.label.value_counts())

Distribution of data based on labels:  1    5362
0    4666
3    2159
4    1937
2    1304
5     572
Name: label, dtype: int64


In [12]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf

In [13]:
from sklearn.model_selection import train_test_split
texts = df["text"].to_list() # Features (not-tokenized yet)
labels = df["label"].to_list() # Lables

# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=0)

# Keep some data for inference (testing)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.01, random_state=0)


In [14]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [13]:
print(train_texts)

['i want to enter in defiance but coming from a different culture i feel offended that i am not allowed', 'im still feeling a little shaky as i write this post', 'i can understand that the people here are not nice to them and that they feel isolated and alone and think this life is just not worth it anymore', 'i still feel embarrassed when i think about it', 'i feel have not convinced me', 'i was not feeling so nervous because she seemed so calm and collected', 'i am feeling hmmmmm melancholy', 'i had a horrible horrible horrible time and honestly this music the monkees was one of the few things that made me feel truly happy and right now i m tearing up which is stupid because yes i am feeling happy', 'i am a mother though most days it still feels strange to realize i am one', 'i have to loathe myself or even allow myself to feel damaged long term', 'i wont be totally satisfied until i feel like me and my work actually means something to more than my loyal reading viewing audience', 'i

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [16]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [17]:
model.fit(train_dataset.shuffle(1000).batch(32), epochs=3, batch_size=32,
          validation_data=val_dataset.shuffle(1000).batch(32))

Epoch 1/3
396/396 [==============================] - 155s 322ms/step - loss: 0.5071 - accuracy: 0.8209 - val_loss: 0.1867 - val_accuracy: 0.9262
Epoch 2/3
396/396 [==============================] - 118s 299ms/step - loss: 0.1480 - accuracy: 0.9383 - val_loss: 0.1587 - val_accuracy: 0.9325
Epoch 3/3
396/396 [==============================] - 118s 298ms/step - loss: 0.1051 - accuracy: 0.9515 - val_loss: 0.1744 - val_accuracy: 0.9322


In [18]:
model.save_pretrained('bertmodel/model')

In [21]:
def get_emotions(text):
    new_text = clean_text(text)
    new_text = new_text

    seq = tokenizer.encode(new_text,truncation=True, padding=True,return_tensors="tf")
    pred = model.predict(seq)
    label_name = ['sadness','joy','love','anger','fear','surprised']
    pred_labels = int(np.argmax(pred.logits, axis=1))
    return label_name[pred_labels]

In [ ]:
def Negation(sentence):	
  '''
  Input: Tokenized sentence (List of words)
  Output: Tokenized sentence with negation handled (List of words)
  '''
  temp = int(0)
  for i in range(len(sentence)):
      if sentence[i-1] in ['not',"n't"]:
          antonyms = []
          for syn in wordnet.synsets(sentence[i]):
              syns = wordnet.synsets(sentence[i])
              w1 = syns[0].name()
              temp = 0
              for l in syn.lemmas():
                  if l.antonyms():
                      antonyms.append(l.antonyms()[0].name())
              max_dissimilarity = 0
              for ant in antonyms:
                  syns = wordnet.synsets(ant)
                  w2 = syns[0].name()
                  syns = wordnet.synsets(sentence[i])
                  w1 = syns[0].name()
                  word1 = wordnet.synset(w1)
                  word2 = wordnet.synset(w2)
                  if isinstance(word1.wup_similarity(word2), float) or isinstance(word1.wup_similarity(word2), int):
                      temp = 1 - word1.wup_similarity(word2)
                  if temp>max_dissimilarity:
                      max_dissimilarity = temp
                      antonym_max = ant
                      sentence[i] = antonym_max
                      sentence[i-1] = ''
  while '' in sentence:
      sentence.remove('')
  return sentence

In [23]:
text = ''
print(get_emotions(text))

1/1 [==============================] - 3s 3s/step
love
